# Combine videos sequentially

In [ ]:
from moviepy.editor import VideoFileClip, concatenate_videoclips
import re

# List of paths to the video files
video_paths = [r"C:\Users\11\Documents\2024-08-11 11-46-26.mp4", r"C:\Users\11\Documents\2024-08-11 11-52-27.mp4"]

# Load the video clips
video_clips = [VideoFileClip(video) for video in video_paths]

# Combine the videos one after another
combined = concatenate_videoclips(video_clips)

# Write the result to a file
combined.write_videofile(r"C:\Users\11\Documents\combined_video.mp4")

# Trim Video duration

In [ ]:
from moviepy.video.io.VideoFileClip import VideoFileClip

def trim_video(input_path, output_path, duration=300):
    """
    Trims the video to the specified duration and saves it to a new file.

    :param input_path: Path to the input video file.
    :param output_path: Path to save the trimmed video file.
    :param duration: Duration in seconds to keep from the start of the video. Default is 300 seconds (5 minutes).
    """
    # Load the video file
    video = VideoFileClip(input_path)
    
    # Trim the video to the first 'duration' seconds
    trimmed_video = video.subclip(0, duration)
    
    # Write the trimmed video to a new file
    trimmed_video.write_videofile(output_path, codec="libx264")


input_video_path = "v1.mp4"
output_video_path = "v1_5min.mp4"
trim_video(input_video_path, output_video_path, 300)

# Export Video frames

In [ ]:
import cv2
import os, re
import time
from concurrent.futures import ThreadPoolExecutor, as_completed

def save_frame(frame, frame_filename):
    cv2.imwrite(frame_filename, frame)

def video_to_frames(video_path, output_folder=None, max_workers=8, frame_skip=0):
    start_time = time.time()
    
    # Get the base name of the video file (without extension)
    video_name = os.path.splitext(os.path.basename(video_path))[0]
    
    # Set the default output folder if not provided
    if output_folder is None:
        output_folder = os.path.join(os.getcwd(), video_name)
    
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # Open the video file
    video_capture = cv2.VideoCapture(video_path)
    
    # Check if the video was opened successfully
    if not video_capture.isOpened():
        print(f"Error: Could not open video {video_path}")
        return
    
    frame_count = 0
    saved_frame_count = 0
    futures = []
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        while True:
            # Read the next frame from the video
            success, frame = video_capture.read()
            
            # If reading a frame was not successful, break the loop
            if not success:
                break
            
            # Save the frame if it's not being skipped
            if frame_count % (frame_skip + 1) == 0:
                # Construct the output file path
                frame_filename = os.path.join(output_folder, f"frame_{saved_frame_count:04d}.png")
                
                # Submit the frame saving task to the thread pool
                futures.append(executor.submit(save_frame, frame, frame_filename))
                
                saved_frame_count += 1
            
            frame_count += 1
    
    # Wait for all futures to complete
    for future in as_completed(futures):
        future.result()
    
    # Release the video capture object
    video_capture.release()
    
    end_time = time.time()
    total_time = end_time - start_time
    print(f"Extracted {saved_frame_count} frames to {output_folder} in {total_time:.2f} seconds")


video_to_frames("v2.mp4", frame_skip=20)
video_to_frames("v3.mp4", frame_skip=20)
video_to_frames("v4.mp4", frame_skip=20)
video_to_frames("v5.mp4", frame_skip=20)

In [ ]:
import winshell
import keyboard

# Function to restore the last deleted item from the Recycle Bin
def restore_last_deleted_item():
    try:
        # Get the list of items in the Recycle Bin
        items = list(winshell.recycle_bin())
        
        if items:
            # Restore the last deleted item
            last_item = items[-1]
            last_item.restore()
            print(f"Successfully restored: {last_item.name}")
        else:
            print("No items found in the Recycle Bin.")
    except Exception as e:
        print(f"Failed to restore the last deleted item: {e}")

# Set up the hotkey to restore the last deleted item
hotkey = 'numlock'
keyboard.add_hotkey(hotkey, restore_last_deleted_item)

print(f"Press {hotkey} to restore the last deleted item from the Recycle Bin.")

# Keep the script running
keyboard.wait('esc')

# CSV to YOLO text annotations

In [ ]:
import os
import pandas as pd

# Read the CSV data
data = pd.read_csv('labels_valomalo_2024-08-09-12-55-25.csv')

# Map class names to class IDs
class_mapping = {
    'Body': 0,
    'Head': 1
}

# Function to normalize coordinates
def normalize_bbox(x, y, width, height, img_width, img_height):
    x_center = (x + width / 2) / img_width
    y_center = (y + height / 2) / img_height
    norm_width = width / img_width
    norm_height = height / img_height
    return x_center, y_center, norm_width, norm_height

# Directory to save the annotation files
output_dir = 'annotations'
os.makedirs(output_dir, exist_ok=True)

# Process each row and write to corresponding text file
for _, row in data.iterrows():
    class_id = class_mapping[row['label_name']]
    x_center, y_center, norm_width, norm_height = normalize_bbox(
        row['bbox_x'], row['bbox_y'], row['bbox_width'], row['bbox_height'],
        row['image_width'], row['image_height']
    )
    
    # Format the annotation to six decimal places
    annotation = f"{class_id} {x_center:.6f} {y_center:.6f} {norm_width:.6f} {norm_height:.6f}\n"
    
    # Write to the corresponding text file in the specified directory
    txt_file_name = os.path.join(output_dir, row['image_name'].replace('.jpg', '.txt'))
    with open(txt_file_name, 'a') as f:
        f.write(annotation)

# Visualize images & their labels + option to extract to a new dir

In [4]:
import os
import random
import cv2
import matplotlib.pyplot as plt

def get_label_color(label, label_colors, predefined_colors):
    if label in predefined_colors:
        return predefined_colors[label]
    if label not in label_colors:
        label_colors[label] = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
    return label_colors[label]

def draw_bounding_boxes(image, label_path, label_colors, predefined_colors, thickness=2):
    height, width, _ = image.shape
    
    if os.path.exists(label_path):
        with open(label_path, 'r') as file:
            for line in file:
                parts = line.strip().split()
                label = int(parts[0])
                x_center, y_center, w, h = map(float, parts[1:])
                
                x_center *= width
                y_center *= height
                w *= width
                h *= height
                
                x1 = int(x_center - w / 2)
                y1 = int(y_center - h / 2)
                x2 = int(x_center + w / 2)
                y2 = int(y_center + h / 2)
                
                color = get_label_color(label, label_colors, predefined_colors)
                cv2.rectangle(image, (x1, y1), (x2, y2), color, thickness)
    return image

def visualize_images_with_labels(image_folder, label_folder, predefined_colors, thickness=2):
    label_colors = {}
    image_files = [f for f in os.listdir(image_folder) if f.endswith(('.png', '.jpg', '.jpeg'))]
    
    for image_file in image_files:
        image_path = os.path.join(image_folder, image_file)
        label_path = os.path.join(label_folder, os.path.splitext(image_file)[0] + '.txt')
        
        image = cv2.imread(image_path)
        image = draw_bounding_boxes(image, label_path, label_colors, predefined_colors, thickness)
        
        # Print the image name to the console
        print(f"{image_file}")
        
        plt.figure(figsize=(10, 10))
        plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        plt.axis('off')
        plt.show()

def save_images_with_labels(image_folder, label_folder, output_folder, predefined_colors, thickness=2):
    label_colors = {}
    image_files = [f for f in os.listdir(image_folder) if f.endswith(('.png', '.jpg', '.jpeg'))]
    
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    for image_file in image_files:
        image_path = os.path.join(image_folder, image_file)
        label_path = os.path.join(label_folder, os.path.splitext(image_file)[0] + '.txt')
        
        image = cv2.imread(image_path)
        image = draw_bounding_boxes(image, label_path, label_colors, predefined_colors, thickness)
        
        # Save the image with bounding boxes to the output folder
        output_path = os.path.join(output_folder, image_file)
        cv2.imwrite(output_path, image)
        
        # Print the image name to the console
        print(f"Saved {image_file} to {output_folder}")

# Example usage
image_folder = r"C:\Users\Vaquita\Downloads\Valomalo\final dataset\images_cutout_fix"
label_folder = r"C:\Users\Vaquita\Downloads\Valomalo\final dataset\labels_cutout_fix"
output_folder = r"C:\Users\Vaquita\Downloads\Valomalo\final dataset\images_with_labels_fix"

# Define predefined colors for labels
predefined_colors = {
    1: (255, 0, 0),    # Red 
    0: (0, 255, 0),    # Green 
    # Add more predefined colors as needed
}

# Visualize images with labels
# visualize_images_with_labels(image_folder, label_folder, predefined_colors, thickness=3)

# Save images with labels to output folder
save_images_with_labels(image_folder, label_folder, output_folder, predefined_colors, thickness=2)

Saved image_1723283693.jpg to C:\Users\Vaquita\Downloads\Valomalo\final dataset\images_with_labels_fix
Saved image_1723283694.jpg to C:\Users\Vaquita\Downloads\Valomalo\final dataset\images_with_labels_fix
Saved image_1723283698.jpg to C:\Users\Vaquita\Downloads\Valomalo\final dataset\images_with_labels_fix
Saved image_1723283699.jpg to C:\Users\Vaquita\Downloads\Valomalo\final dataset\images_with_labels_fix
Saved image_1723283706.jpg to C:\Users\Vaquita\Downloads\Valomalo\final dataset\images_with_labels_fix
Saved image_1723283707.jpg to C:\Users\Vaquita\Downloads\Valomalo\final dataset\images_with_labels_fix
Saved image_1723283708.jpg to C:\Users\Vaquita\Downloads\Valomalo\final dataset\images_with_labels_fix
Saved image_1723283719.jpg to C:\Users\Vaquita\Downloads\Valomalo\final dataset\images_with_labels_fix
Saved image_1723283740.jpg to C:\Users\Vaquita\Downloads\Valomalo\final dataset\images_with_labels_fix
Saved image_1723283741.jpg to C:\Users\Vaquita\Downloads\Valomalo\final d

# Extract a Cutout of the Image containing the Labels + save the new adjust labels

In [3]:
import os
import random
from PIL import Image

def extract_cutout(image_path, label_path, label_number, cutout_size):
    image = Image.open(image_path)
    width, height = image.size
    cutout_width, cutout_height = cutout_size

    # Ensure cutout size is not larger than the image dimensions
    cutout_width = min(cutout_width, width)
    cutout_height = min(cutout_height, height)

    labels = []
    target_labels = []

    # If label file exists, process it
    if os.path.exists(label_path):
        with open(label_path, 'r') as file:
            lines = file.readlines()
            for line in lines:
                parts = line.strip().split()
                label = int(parts[0])
                x_center = float(parts[1]) * width
                y_center = float(parts[2]) * height
                box_width = float(parts[3]) * width
                box_height = float(parts[4]) * height

                if label == label_number:
                    target_labels.append((label, x_center, y_center, box_width, box_height))

            if target_labels:
                # Calculate the bounding box that includes all target labels
                min_left = min(x_center - box_width / 2 for _, x_center, _, box_width, _ in target_labels)
                max_right = max(x_center + box_width / 2 for _, x_center, _, box_width, _ in target_labels)
                min_top = min(y_center - box_height / 2 for _, _, y_center, _, box_height in target_labels)
                max_bottom = max(y_center + box_height / 2 for _, _, y_center, _, box_height in target_labels)

                # Randomly position the cutout around the bounding box
                left = int(min_left - random.uniform(0.25, 0.75) * (cutout_width - (max_right - min_left)))
                top = int(min_top - random.uniform(0.25, 0.75) * (cutout_height - (max_bottom - min_top)))
                right = left + cutout_width
                bottom = top + cutout_height

                # Ensure the cutout is within image bounds
                left = max(0, min(left, width - cutout_width))
                top = max(0, min(top, height - cutout_height))
                right = left + cutout_width
                bottom = top + cutout_height

                cutout = image.crop((left, top, right, bottom))

                # Adjust all label coordinates within the cutout region
                for line in lines:
                    parts = line.strip().split()
                    label = int(parts[0])
                    x_center = float(parts[1]) * width
                    y_center = float(parts[2]) * height
                    box_width = float(parts[3]) * width
                    box_height = float(parts[4]) * height

                    if left <= x_center <= right and top <= y_center <= bottom:
                        new_x_center = (x_center - left) / cutout_width
                        new_y_center = (y_center - top) / cutout_height
                        new_box_width = box_width / cutout_width
                        new_box_height = box_height / cutout_height

                        # Clip the values to be within [0, 1]
                        new_x_center = min(max(new_x_center, 0), 1)
                        new_y_center = min(max(new_y_center, 0), 1)
                        new_box_width = min(max(new_box_width, 0), 1)
                        new_box_height = min(max(new_box_height, 0), 1)

                        labels.append((label, new_x_center, new_y_center, new_box_width, new_box_height))
                return cutout, labels

            # If no target labels found, use the next available label
            for line in lines:
                parts = line.strip().split()
                label = int(parts[0])
                x_center = float(parts[1]) * width
                y_center = float(parts[2]) * height
                box_width = float(parts[3]) * width
                box_height = float(parts[4]) * height

                # Randomly position the cutout around the label
                left = int(x_center - random.uniform(0.25, 0.75) * cutout_width)
                top = int(y_center - random.uniform(0.25, 0.75) * cutout_height)
                right = left + cutout_width
                bottom = top + cutout_height

                # Ensure the cutout is within image bounds
                left = max(0, min(left, width - cutout_width))
                top = max(0, min(top, height - cutout_height))
                right = left + cutout_width
                bottom = top + cutout_height

                cutout = image.crop((left, top, right, bottom))

                # Adjust all label coordinates within the cutout region
                for line in lines:
                    parts = line.strip().split()
                    label = int(parts[0])
                    x_center = float(parts[1]) * width
                    y_center = float(parts[2]) * height
                    box_width = float(parts[3]) * width
                    box_height = float(parts[4]) * height

                    if left <= x_center <= right and top <= y_center <= bottom:
                        new_x_center = (x_center - left) / cutout_width
                        new_y_center = (y_center - top) / cutout_height
                        new_box_width = box_width / cutout_width
                        new_box_height = box_height / cutout_height

                        # Clip the values to be within [0, 1]
                        new_x_center = min(max(new_x_center, 0), 1)
                        new_y_center = min(max(new_y_center, 0), 1)
                        new_box_width = min(max(new_box_width, 0), 1)
                        new_box_height = min(max(new_box_height, 0), 1)

                        labels.append((label, new_x_center, new_y_center, new_box_width, new_box_height))
                return cutout, labels
    else:
        # If no labels, extract a random region from the middle
        if width > cutout_width and height > cutout_height:
            left_min = max(0, width // 4)
            left_max = min(width - cutout_width, 3 * width // 4)
            top_min = max(0, height // 4)
            top_max = min(height - cutout_height, 3 * height // 4)

            left = random.randint(left_min, left_max)
            top = random.randint(top_min, top_max)
        else:
            left = (width - cutout_width) // 2
            top = (height - cutout_height) // 2

        right = left + cutout_width
        bottom = top + cutout_height

        cutout = image.crop((left, top, right, bottom))
        return cutout, labels

def process_images(image_folder, label_folder, label_number, cutout_size, output_folder, label_output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    if not os.path.exists(label_output_folder):
        os.makedirs(label_output_folder)

    for image_name in os.listdir(image_folder):
        if image_name.endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(image_folder, image_name)
            label_path = os.path.join(label_folder, os.path.splitext(image_name)[0] + '.txt')
            cutout, labels = extract_cutout(image_path, label_path, label_number, cutout_size)
            if cutout:
                cutout.save(os.path.join(output_folder, image_name))
                label_output_path = os.path.join(label_output_folder, os.path.splitext(image_name)[0] + '.txt')
                with open(label_output_path, 'w') as label_file:
                    if labels:
                        for label in labels:
                            label_file.write(f"{label[0]} {label[1]:.6f} {label[2]:.6f} {label[3]:.6f} {label[4]:.6f}\n")
                    else:
                        # Write an empty file for images with no labels
                        label_file.write("")

image_folder = r"C:\Users\Vaquita\Downloads\Valomalo\final dataset\images"
label_folder = r"C:\Users\Vaquita\Downloads\Valomalo\final dataset\labels"
label_number = 0 # The label to choose the cutouts upon
cutout_size = (640, 640)  
image_output_folder = r"C:\Users\Vaquita\Downloads\Valomalo\final dataset\images_cutout_fix"
label_output_folder = r"C:\Users\Vaquita\Downloads\Valomalo\final dataset\labels_cutout_fix"

process_images(image_folder, label_folder, label_number, cutout_size, image_output_folder, label_output_folder)


# Split the main folder into TvT

In [14]:
import os, re
import random
import shutil

images_path = r"C:\Users\Vaquita\Downloads\Valomalo\final dataset\images_cutout"
labels_path = r"C:\Users\Vaquita\Downloads\Valomalo\final dataset\labels_cutout"

# Preprocessing step: Delete .txt files that do not correspond to any image
label_files = [f for f in os.listdir(labels_path) if f.endswith('.txt')]
for label_file in label_files:
    image_file = label_file.replace('.txt', '.jpg')
    if not os.path.exists(os.path.join(images_path, image_file)):
        os.remove(os.path.join(labels_path, label_file))

# Get all image files
image_files = [f for f in os.listdir(images_path) if f.endswith('.jpg')]

# Shuffle the files
random.shuffle(image_files)

# Calculate split indices
total_images = len(image_files)
train_split = int(total_images * 0.7)
val_split = int(total_images * 0.2)

# Split the files
train_files = image_files[:train_split]
val_files = image_files[train_split:train_split + val_split]
test_files = image_files[train_split + val_split:]

# Copy files to their directories
def copy_files(file_list, dest_images_path, dest_labels_path):
    os.makedirs(dest_images_path, exist_ok=True)
    os.makedirs(dest_labels_path, exist_ok=True)
    for file in file_list:
        shutil.copy(os.path.join(images_path, file), dest_images_path)
        label_file = file.replace('.jpg', '.txt')
        if os.path.exists(os.path.join(labels_path, label_file)):
            shutil.copy(os.path.join(labels_path, label_file), dest_labels_path)

# Create directories and copy files
copy_files(train_files, 'yolo_dataset/train/images', 'yolo_dataset/train/labels')
copy_files(val_files, 'yolo_dataset/val/images', 'yolo_dataset/val/labels')
copy_files(test_files, 'yolo_dataset/test/images', 'yolo_dataset/test/labels')

# Copy images with no Labels into a new dir

In [2]:
import os, re
import shutil

def copy_images_without_labels(images_dir, labels_dir, dest_dir):
    # Ensure the destination directory exists
    os.makedirs(dest_dir, exist_ok=True)

    # Get the list of image files and label files
    image_files = [f for f in os.listdir(images_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]
    label_files = [f for f in os.listdir(labels_dir) if f.endswith('.txt')]

    # Create a set of label filenames without extensions
    label_names = set(os.path.splitext(f)[0] for f in label_files)

    # Iterate through image files and copy those without corresponding labels
    for image_file in image_files:
        image_name, _ = os.path.splitext(image_file)
        if image_name not in label_names:
            src_path = os.path.join(images_dir, image_file)
            dest_path = os.path.join(dest_dir, image_file)
            shutil.copy(src_path, dest_path) # comment if you dont wanna copy or add logic
            print(f"Copied {image_file} to {dest_dir}")

images_directory = r'C:\Users\Vaquita\Downloads\Valomalo\final dataset\images'
labels_directory = r'C:\Users\Vaquita\Downloads\Valomalo\final dataset\labels'
destination_directory = r'C:\Users\Vaquita\Downloads\Valomalo\final dataset\no_labels'

copy_images_without_labels(images_directory, labels_directory, destination_directory)

# Find closest match of images to the images in the bigger dir using an image hash

In [ ]:
from PIL import Image
import imagehash
import os
from matplotlib import pyplot as plt

def load_images_from_folder(folder):
    images = []
    filenames = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        try:
            img = Image.open(img_path)
            images.append(img)
            filenames.append(filename)
        except Exception as e:
            print(f"Error loading image {filename}: {e}")
    return images, filenames

def compute_hash(image):
    return imagehash.average_hash(image)

def find_closest_image(hash_x, hashes_y):
    min_distance = float('inf')
    closest_index = -1
    for i, hash_y in enumerate(hashes_y):
        distance = hash_x - hash_y
        if distance < min_distance:
            min_distance = distance
            closest_index = i
    return closest_index, min_distance

def find_closest_images(dir_x, dir_y, threshold=float('inf'), display_results=False):
    images_x, filenames_x = load_images_from_folder(dir_x)
    images_y, filenames_y = load_images_from_folder(dir_y)

    hashes_x = [compute_hash(img) for img in images_x]
    hashes_y = [compute_hash(img) for img in images_y]

    results = []

    for i, hash_x in enumerate(hashes_x):
        closest_index, min_distance = find_closest_image(hash_x, hashes_y)
        if min_distance <= threshold:
            result = {
                'image_x': filenames_x[i],
                'closest_image_y': filenames_y[closest_index],
                'difference_score': min_distance
            }
            results.append(result)
            print(f"Image {filenames_x[i]} from X is closest to {filenames_y[closest_index]} from Y with a difference score of {min_distance}")

            if display_results:
                # Display the images
                plt.figure(figsize=(10, 5))
                plt.subplot(1, 2, 1)
                plt.title(f"Image from X: {filenames_x[i]}")
                plt.imshow(images_x[i])
                plt.axis('off')

                plt.subplot(1, 2, 2)
                plt.title(f"Closest Image from Y: {filenames_y[closest_index]}")
                plt.imshow(images_y[closest_index])
                plt.axis('off')

                plt.show()

    return results

dir_images_to_find = r'C:\Users\Vaquita\Downloads\Valomalo\test_benchmark_images'
dir_images_to_search = r"C:\Users\Vaquita\Downloads\Valomalo\test_benchmark_images"

threshold = 1  # Set your desired threshold here


results = find_closest_images(dir_images_to_find, dir_images_to_search, threshold=threshold, display_results=True)

In [ ]:
# Store all the closest matches in a list
closest_matches = results

# Print the closest matches
for match in closest_matches:
    print(f"{match['closest_image_y']} ")

# copy files in a list into a new folder

In [ ]:
import shutil
import os

def copy_files(file_list, source_dir, destination_dir):
    """
    Copies files from the source directory to the destination directory.

    :param file_list: List of filenames to be copied.
    :param source_dir: Directory where the files are currently located.
    :param destination_dir: Directory where the files should be copied to.
    """
    if not os.path.exists(destination_dir):
        os.makedirs(destination_dir)

    for filename in file_list:
        src_path = os.path.join(source_dir, filename)
        dest_path = os.path.join(destination_dir, filename)
        try:
            shutil.copy(src_path, dest_path)
            print(f"Copied {filename} to {destination_dir}")
        except Exception as e:
            print(f"Error copying {filename}: {e}")

source_dir = r'C:\Users\Vaquita\Downloads\Valomalo\Moamen - batch 1'
destination_dir = r'C:\Users\Vaquita\Downloads\Valomalo\Moamen - batch 1\copied_files'


# Extract the list of closest image filenames from Y
closest_image_filenames = [match['closest_image_y'] for match in closest_matches]

# Copy the closest image files to the new folder
copy_files(closest_image_filenames, source_dir, destination_dir)


# Visualize YOLO Augmentation

## Approved Augs
A.HorizontalFlip(p=0.5),
A.RandomScale(scale_limit=0.2, p=0.5),  # Randomly scale the image with a 50% probability
A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.2, rotate_limit=20, p=1),  # Randomly shift, scale, and rotate the image with a 50% probability
A.CoarseDropout(max_holes=8, max_height=128, max_width=128, p=1),  # Randomly drop rectangular regions of the image with a 50% probability


In [ ]:
import os
import random
import cv2
import matplotlib.pyplot as plt
import albumentations as A

# Define paths
image_folder = r'C:\Users\Vaquita\Downloads\Valomalo\Moamen - batch 1'
label_folder = r'C:\Users\Vaquita\Downloads\Valomalo\Moamen - batch 1\labels_my-project-name_2024-08-14-10-47-42'

# Function to read YOLO format labels
def read_yolo_labels(label_path):
    with open(label_path, 'r') as file:
        labels = [line.strip().split() for line in file]
    return labels

# Function to draw bounding boxes on the image
def draw_boxes(image, boxes, color=(0, 255, 0)):
    h, w, _ = image.shape
    for box in boxes:
        class_id, x_center, y_center, width, height = map(float, box)
        x1 = int((x_center - width / 2) * w)
        y1 = int((y_center - height / 2) * h)
        x2 = int((x_center + width / 2) * w)
        y2 = int((y_center + height / 2) * h)
        cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
    return image

# Define augmentation pipeline
transform = A.Compose([
    #A.HorizontalFlip(p=0.5),  # Flip the image horizontally with a 50% probability
    A.RandomScale(scale_limit=0.5, p=1),  # Randomly scale the image with a 50% probability
    #A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.2, rotate_limit=20, p=1),  # Randomly shift, scale, and rotate the image with a 50% probability
    #A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2, p=0.5),  # Randomly change brightness, contrast, saturation, and hue with a 50% probability
    #A.CoarseDropout(max_holes=8, max_height=128, max_width=128, p=1),  # Randomly drop rectangular regions of the image with a 50% probability
], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))

# Get list of images and labels
image_files = [f for f in os.listdir(image_folder) if f.endswith('.jpg')]
label_files = [f for f in os.listdir(label_folder) if f.endswith('.txt')]

# Select random images for visualization
random_images = random.sample(image_files, 5)

for image_file in random_images:
    # Read image
    image_path = os.path.join(image_folder, image_file)
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Read corresponding label
    label_path = os.path.join(label_folder, os.path.splitext(image_file)[0] + '.txt')
    labels = read_yolo_labels(label_path)
    
    # Extract bounding boxes and class labels
    bboxes = [list(map(float, label[1:])) for label in labels]
    class_labels = [label[0] for label in labels]
    
    # Apply augmentation
    augmented = transform(image=image, bboxes=bboxes, class_labels=class_labels)
    augmented_image = augmented['image']
    augmented_bboxes = augmented['bboxes']
    
    # Draw bounding boxes on original and augmented images
    image_with_boxes = draw_boxes(image.copy(), labels)
    augmented_image_with_boxes = draw_boxes(augmented_image.copy(), [[class_labels[i]] + list(bbox) for i, bbox in enumerate(augmented_bboxes)])
    
    # Plot the images
    fig, ax = plt.subplots(1, 2, figsize=(12, 6))
    ax[0].imshow(image_with_boxes)
    ax[0].set_title('Original Image')
    ax[1].imshow(augmented_image_with_boxes)
    ax[1].set_title('Augmented Image')
    plt.show()

# Verify corrct annoations for YOLO

In [7]:
import os

def check_yolo_annotations(directory):
    """
    Reads all YOLO annotation .txt files in the specified directory and checks if any of the coordinates
    are outside the range [0, 1]. Prints the file names with errors and their annotations.
    
    :param directory: The directory containing YOLO annotation .txt files.
    """
    error_count = 0

    for filename in os.listdir(directory):
        if filename.endswith(".txt"):
            filepath = os.path.join(directory, filename)
            with open(filepath, 'r') as file:
                lines = file.readlines()
                for line in lines:
                    parts = line.strip().split()
                    if len(parts) != 5:
                        print(f"Error in file {filename}: Incorrect annotation format: {line.strip()}")
                        error_count += 1
                        continue
                    _, x_center, y_center, width, height = map(float, parts)
                    if not (0 <= x_center <= 1 and 0 <= y_center <= 1 and 0 <= width <= 1 and 0 <= height <= 1):
                        print(f"Error in file {filename}: {line.strip()}")
                        error_count += 1

    print(f"Total number of errors: {error_count}")

# Example usage
check_yolo_annotations(r'C:\Users\Vaquita\Downloads\Valomalo\final dataset\labels_cutout_fix')

Total number of errors: 0
